In [1]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /opt/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/h0/4hznbs6x77gf8d5h9lp1hkwr0000gn/T/tmp38bn8bal
  JVM stdout: /var/folders/h0/4hznbs6x77gf8d5h9lp1hkwr0000gn/T/tmp38bn8bal/h2o_julio_started_from_python.out
  JVM stderr: /var/folders/h0/4hznbs6x77gf8d5h9lp1hkwr0000gn/T/tmp38bn8bal/h2o_julio_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_julio_b7bqyk
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.778 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [2]:
import pandas as pd

In [3]:
diamonds=pd.read_csv('diamonds_train.csv', index_col=0)
df=h2o.H2OFrame(diamonds)


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
df=df.asnumeric()

In [7]:
df.columns

['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z', 'price']

In [8]:
y_columns="price"
x_columns=['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']

In [9]:
train,test=df.split_frame(ratios=[.8])

In [10]:
x_train=train[x_columns]
y_trin=train[y_columns]
x_test=test[x_columns]
y_test=test[y_columns]

In [11]:
from h2o.automl import H2OAutoML

In [12]:
aml_ti=H2OAutoML(max_runtime_secs=180,max_models=40,seed=1,nfolds=0)
aml_ti.train(x=x_columns,y=y_columns,training_frame=train,validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [25]:
lb_ti=aml_ti.leaderboard
lb_ti

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_2_AutoML_20200511_145955,259334,509.248,259334,273.871,0.102217
GBM_4_AutoML_20200511_145955,262083,511.94,262083,263.645,0.0911386
GBM_3_AutoML_20200511_145955,262930,512.767,262930,270.351,0.101284
GBM_grid__1_AutoML_20200511_145955_model_1,266079,515.828,266079,272.805,0.0977065
GBM_1_AutoML_20200511_145955,273327,522.806,273327,281.18,0.102262
XGBoost_grid__1_AutoML_20200511_145955_model_2,280076,529.222,280076,274.413,0.0894725
XGBoost_grid__1_AutoML_20200511_145955_model_3,286226,535.001,286226,280.029,0.0989744
XGBoost_grid__1_AutoML_20200511_145955_model_6,291258,539.683,291258,277.568,0.0925669
XGBoost_3_AutoML_20200511_145955,298665,546.503,298665,293.312,0.103615
XGBoost_grid__1_AutoML_20200511_145955_model_1,302642,550.129,302642,284.475,0.0950268


In [31]:
test = pd.read_csv('diamonds_test.csv', index_col=0)
test=h2o.H2OFrame(test)


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [16]:
test=test.asnumeric()

In [32]:
preds=aml_ti.leader.predict(x)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [33]:
preds2=preds.as_data_frame(use_pandas=True)

In [34]:
preds2

,predict
0,1864.726979
1,580.174055
2,2438.562769
3,5877.234835
4,8480.870733
...,...
13444,1006.063294
13445,2169.292781
13446,1981.704159
13447,2257.580373


In [35]:
test=df2.as_data_frame(use_pandas=True)

In [36]:
test

,carat,cut,color,clarity,depth,table,x,y,z
0,0.56,Ideal,G,VS1,60.6,56.0,5.33,5.36,3.24
1,0.32,Premium,H,VS2,61.9,58.0,4.39,4.37,2.71
2,0.71,Good,G,VS2,64.2,58.0,5.59,5.62,3.60
3,1.12,Premium,H,VS1,62.1,58.0,6.64,6.60,4.11
4,1.21,Ideal,G,VS2,61.5,56.0,6.83,6.88,4.22
...,...,...,...,...,...,...,...,...,...
13444,0.42,Ideal,H,VS1,61.2,57.0,4.87,4.80,2.96
13445,0.71,Fair,H,VS1,66.4,62.0,5.54,5.49,3.66
13446,0.60,Ideal,G,VS1,61.1,57.0,5.41,5.45,3.32
13447,0.70,Premium,D,SI2,62.8,61.0,5.61,5.57,3.51


In [37]:
results=pd.DataFrame()
results["id"]=range(len(preds2))
results["price"]=preds2["predict"]

In [38]:
results.to_csv("my_submition13",index=False)